# Webscraping Project on Flight Price Prediction

In [52]:
import selenium
import pandas as pd
import numpy as np
import time
import requests
import re
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

In [53]:
# Connecting to the webdriver 
driver = webdriver.Chrome(r"C:\Users\shaha\Downloads\chromedriver_win32 (1)/chromedriver.exe")
driver.maximize_window() 

In [54]:
# Getting the webpage of mentioned url
url = "https://www.yatra.com/"
driver.get(url)
time.sleep(3)

In [55]:
#driver.close()

In [56]:
##clicking search button
search_btn = driver.find_element_by_xpath("//input[@class='js_ripple search-btn ']")
search_btn.click()

# Now we will scarpe data from website

In [57]:
# Let's make a list of locations to scrap data
location_lst = ['New Delhi', 'Mumbai','Pune', 'Bangalore', 'Chennai','Hyderabad ','Goa ','Kolkata ','Jaipur ','Lucknow ']

In [58]:
#making empty to store data:
Airline = []
Source = []
Destination = []
Departure_Time = []
Arrival_Time = []
Duration = []
Meal=[] 
Total_Stops = []
Price = []
Location=[]

In [59]:
# Scraping the data from the mentioned url

# Fetching webelement of source 
for x in location_lst:
    for y in location_lst:
        if x!=y:                 
            Source = driver.find_element_by_xpath("//div[@class='input-holder pb-2 bdr-btm']/input")
            time.sleep(2)
            Source.clear() 
            Source.send_keys(x)
            time.sleep(2)
            
            # Fetching webelement of destination
            Destination = driver.find_element_by_xpath("//div[@class='input-holder  bdr-btm pb-2']/input")
            Destination.click()
            Destination.clear()
            time.sleep(2)
            Destination.send_keys(y)
            time.sleep(2)
            
            try:
                driver.find_element_by_xpath("/html/body/section[2]/section/div[1]/div/div[1]").click()
            except NoSuchElementException:
                pass
            # Searching for flights again and clicking in search again button
            try:
                srch_btn = driver.find_element_by_xpath("//button[@class='fs-14 btn-submit cursor-pointer bold']")
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of(srch_btn))
                srch_btn.click()
            except NoSuchElementException:
                pass
            time.sleep(2)
            
            # Scrolling the data 5000 times 
            for _ in range(15):
                time.sleep(1)
                driver.execute_script("window.scrollBy(0,5000)")
                time.sleep(5)
            
            # Fetching web element of scroll to top button
            try:
                driver.find_element_by_xpath("//div[@title='scroll to top']").click()
            except NoSuchElementException:
                pass
            
           # Fetching name of Airline
            try:
                for i in driver.find_elements_by_xpath("//div[@class='fs-13 airline-name no-pad col-8']/span"):
                    Airline.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching departure time of the flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pr']"):
                    Departure_Time.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching arrival time of the flights 
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pdd-0 text-left atime col-5']//p[1]"):
                    Arrival_Time.append(i.get_attribute("innerHTML").splitlines()[0][0:5])
            except NoSuchElementException:
                pass
            
            # Fetching Duration of flights journey
            try:
                for i in driver.find_elements_by_xpath("//div[@class='stop-cont pl-13']/p"):
                    Duration.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching number of stops the flights have between source and destination
            try:
                for i in driver.find_elements_by_xpath("//div[@class=' font-lightgrey fs-10 tipsy i-b fs-10']/span[1]"):
                    Total_Stops.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching availability of meal in flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='features pull-right fs-12 flex']/div[1]"):
                    Meal.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching Locations
            try:
                for i in driver.find_elements_by_xpath("//p[@class='fs-10 font-lightgrey no-wrap city ellipsis']"):
                    Location.append(i.text)
            except NoSuchElementException:
                pass
            
            # Fetching Prices of flight tickets
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b tipsy fare-summary-tooltip fs-18']"):
                    Price.append(i.text)
            except NoSuchElementException:
                pass    

In [60]:
Source = Location[0:len(Location):2]
Destination = Location[1:len(Location):2]

In [61]:
# Checking length of source and destination
len(Source), len(Destination)

(3980, 3980)

In [70]:
# Creating a dataframe for scraped data
data = list(zip(Airline,Departure_Time,Arrival_Time,Duration,Source,Destination,Meal,Total_Stops,Price))
df = pd.DataFrame(data, columns = ["Airline",
                                   "Departure_Time",
                                   "Arrival_Time",
                                   "Duration", 
                                   "Source",
                                   "Destination",
                                   "Meal_availability",
                                   "Total_Stops",
                                   "Price"])

In [71]:
df

,Airline,Departure_Time,Arrival_Time,Duration,Source,Destination,Meal_availability,Total_Stops,Price
0,Go First,15:00,17:00,2h 00m,New Delhi,Mumbai,eCash 250,Non Stop,"7,320"
1,Air India,18:00,20:00,2h 00m,New Delhi,Mumbai,Free Meal,Non Stop,"7,320"
2,Air India,07:00,09:05,2h 05m,New Delhi,Mumbai,Free Meal,Non Stop,"7,320"
3,Vistara,21:50,23:55,2h 05m,New Delhi,Mumbai,Free Meal,Non Stop,"7,320"
4,Air Asia,04:25,06:35,2h 10m,New Delhi,Mumbai,Emissions: 140 Kg CO2,Non Stop,"7,320"
...,...,...,...,...,...,...,...,...,...
3537,Air India,08:40,17:20,8h 40m,Jaipur,Goa,eCash 250,1 Stop,"10,907"
3538,Air India,14:15,23:00,8h 45m,Jaipur,Goa,eCash 250,1 Stop,"10,907"
3539,Air India,06:20,15:15,8h 55m,Jaipur,Goa,eCash 250,1 Stop,"10,907"
3540,Air India,04:50,13:55,9h 05m,Jaipur,Goa,eCash 250,1 Stop,"10,907"


In [72]:
df.shape

(3542, 9)

In [73]:
df.to_excel("Flight_Prices.xlsx")

# !!!!! THANK YOU !!!!!